## LightGBM test

In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

In [79]:
ALL_PITCH = 'intermediate/all_pitch_1.f'

In [80]:
all_pitch = pd.read_feather(ALL_PITCH)
all_pitch.shape

(778767, 87)

### 不要な列を削除

In [81]:
all_pitch.drop(
    columns=[
        '年度', 
        '試合ID', 
        '日付', '時刻', 
        'ホームチームID', 'アウェイチームID', 
        '球場ID', '球場名', 
        '試合種別詳細', '表裏', 
        '投手ID', '投手チームID', '投手投球左右', '投手役割', 
        '打者ID', '打者チームID', '打者打席左右', '打者守備位置',
        'プレイ前走者状況', '一塁走者ID', '二塁走者ID', '三塁走者ID', 
        '捕手ID', '一塁手ID', '二塁手ID', '三塁手ID', '遊撃手ID', '左翼手ID', '中堅手ID', '右翼手ID', 
        '成績対象投手ID', '成績対象打者ID',
        'opening_date', 'game_date',
        'start_time', 'game_time', 'elapsed_time'
    ], inplace=True)

### Rename

In [82]:
all_pitch.rename(columns={
    'データ内連番': 'No',
    '投球位置区域': 'pitch_area',
    '試合内連番': 'No_in_game',
    '試合内投球数': 'pitch_cnt_in_game',
    'イニング': 'inning',
    'イニング内打席数': 'bat_cnt_in_inning',
    '打席内投球数': 'pitch_cnt_in_bat',
    '投手登板順': 'pitch_order',
    '投手試合内対戦打者数': 'player_cnt_in_game',
    '投手試合内投球数': 'pitch_cnt_in_game',
    '投手イニング内投球数': 'pitch_cnt_in_inning',
    '打者打順': 'bat_order',
    '打者試合内打席数': 'bat_cnt_in_game',
    'プレイ前ホームチーム得点数': 'home_point',
    'プレイ前アウェイチーム得点数': 'away_point',
    'プレイ前アウト数': 'out_cnt',
    'プレイ前ボール数': 'ball_cnt',
    'プレイ前ストライク数': 'strike_cnt',
}, inplace=True)

In [83]:
print(all_pitch.shape)
all_pitch.head()

(778767, 50)


,No,pitch_area,No_in_game,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitch_cnt_in_game,pitch_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,first,second,third,base_cnt,top_bot,role,date_from_opening,elapsed_min,ball_0_straight,ball_1_curve,ball_2_slider,ball_3_shoot,ball_4_fork,ball_5_changeup,ball_6_sinker,ball_7_cutball,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,ball_cnt_2-0,ball_cnt_2-1,ball_cnt_2-2,ball_cnt_2-3,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R
0,0,0.0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0,0,0.000000,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,8.0,2,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,0.0,0.0,0.0,1.0,1.0,0,0.200000,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
2,2,5.0,3,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0.0,0.0,0.0,0.0,1.0,1.0,0,0.383333,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
3,3,12.0,4,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0.0,0.0,0.0,0.0,1.0,1.0,0,1.016667,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,4,8.0,5,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0.0,0.0,0.0,0.0,1.0,1.0,0,1.200000,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


### train

In [84]:
train = all_pitch.dropna(subset=['pitch_area'])
train.shape

(257117, 50)

### test

In [85]:
test = all_pitch[all_pitch['pitch_area'].isnull()]
test.shape

(521650, 50)

In [86]:
del all_pitch
gc.collect()

168

In [87]:
train_d = train.drop([
    'No', 
    'pitch_area', 
    'ball_0_straight', 
    'ball_1_curve', 
    'ball_2_slider', 
    'ball_3_shoot', 
    'ball_4_fork', 
    'ball_5_changeup', 
    'ball_6_sinker', 
    'ball_7_cutball'
], axis=1)

In [88]:
test_d = test.drop([
    'No', 
    'pitch_area', 
    'ball_0_straight', 
    'ball_1_curve', 
    'ball_2_slider', 
    'ball_3_shoot', 
    'ball_4_fork', 
    'ball_5_changeup', 
    'ball_6_sinker', 
    'ball_7_cutball'
], axis=1)

## 0_ストレート

In [89]:
lgb_train = lgb.Dataset(train_d, train['ball_0_straight'])

### LGBM学習

In [90]:
%%time
lgb_param = {
        'objective' : 'binary',
        'boosting_type': 'gbdt',
        'metric' : 'binary_logloss',
        'num_leaves' : 31,
        'seed' : 0,
        'learning_rate' : 0.1,
    }

# 学習
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=8000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['binary_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
print('Best CV score:', cv_results['binary_logloss-mean'][-1])

[100]	cv_agg's binary_logloss: 0.668647 + 0.000324434
[200]	cv_agg's binary_logloss: 0.66643 + 0.000322641
[300]	cv_agg's binary_logloss: 0.665142 + 0.000310527
[400]	cv_agg's binary_logloss: 0.664363 + 0.000375507
[500]	cv_agg's binary_logloss: 0.664028 + 0.000421019
[600]	cv_agg's binary_logloss: 0.663781 + 0.000454686
[700]	cv_agg's binary_logloss: 0.663604 + 0.000492428
[800]	cv_agg's binary_logloss: 0.663458 + 0.000358764
[900]	cv_agg's binary_logloss: 0.663544 + 0.000484709
Best num_boost_round: 801
Best CV score: 0.6634557237452842
CPU times: user 3min 58s, sys: 3.02 s, total: 4min 1s
Wall time: 1min 3s


In [91]:
best_iter = int(num_boost_round * 1.1)

### 全体で再学習

In [92]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 1min 5s, sys: 668 ms, total: 1min 6s
Wall time: 17 s


### Feature Importance

In [93]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(10)

,feat_name,feat_imp
6,player_cnt_in_game,873
9,bat_order,1144
11,home_point,1212
12,away_point,1377
8,pitch_cnt_in_inning,1575
1,pitch_cnt_in_game,1737
0,No_in_game,2022
7,pitch_cnt_in_game,2284
23,elapsed_min,2551
22,date_from_opening,3791


In [94]:
%%time
test['ball_0_straight'] = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 33.2 s, sys: 429 ms, total: 33.6 s
Wall time: 8.9 s


## 1_カーブ

In [96]:
%%time
lgb_train = lgb.Dataset(train_d, train['ball_1_curve'])
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)
test['ball_1_curve'] = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 1min 35s, sys: 1.09 s, total: 1min 36s
Wall time: 25.1 s


## 2_スライダー

In [97]:
%%time
lgb_train = lgb.Dataset(train_d, train['ball_2_slider'])
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)
test['ball_2_slider'] = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 1min 38s, sys: 1.31 s, total: 1min 39s
Wall time: 26.4 s


## 3_シュート

In [98]:
%%time
lgb_train = lgb.Dataset(train_d, train['ball_3_shoot'])
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)
test['ball_3_shoot'] = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 1min 38s, sys: 1.36 s, total: 1min 39s
Wall time: 26.3 s


## 4_フォーク

In [99]:
%%time
lgb_train = lgb.Dataset(train_d, train['ball_4_fork'])
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)
test['ball_4_fork'] = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 1min 41s, sys: 1.21 s, total: 1min 43s
Wall time: 26.8 s


## 5_チェンジアップ

In [100]:
%%time
lgb_train = lgb.Dataset(train_d, train['ball_5_changeup'])
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)
test['ball_5_changeup'] = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 1min 45s, sys: 1.32 s, total: 1min 46s
Wall time: 27.9 s


## 6_シンカー

In [101]:
%%time
lgb_train = lgb.Dataset(train_d, train['ball_6_sinker'])
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)
test['ball_6_sinker'] = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 1min 49s, sys: 1.27 s, total: 1min 50s
Wall time: 28.8 s


## 7_カットボール

In [102]:
%%time
lgb_train = lgb.Dataset(train_d, train['ball_7_cutball'])
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)
test['ball_7_cutball'] = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 1min 41s, sys: 1.18 s, total: 1min 42s
Wall time: 26.6 s


## 結果出力

In [115]:
submit = test[
    ['No', 
    'ball_0_straight', 
    'ball_1_curve', 
    'ball_2_slider', 
    'ball_3_shoot', 
    'ball_4_fork', 
    'ball_5_changeup', 
    'ball_6_sinker', 
    'ball_7_cutball']].copy()

In [116]:
submit['total'] = submit['ball_0_straight'] + submit['ball_1_curve'] + submit['ball_2_slider'] + submit['ball_3_shoot'] + submit['ball_4_fork'] + submit['ball_5_changeup'] + submit['ball_6_sinker'] + submit['ball_7_cutball']
submit.head()

,No,ball_0_straight,ball_1_curve,ball_2_slider,ball_3_shoot,ball_4_fork,ball_5_changeup,ball_6_sinker,ball_7_cutball,total
257117,0,0.778962,0.001299,0.055267,0.090204,0.012620,0.002367,0.005786,0.044506,0.991013
257118,1,0.687431,0.009419,0.049380,0.095945,0.013453,0.005128,0.000532,0.031697,0.892985
257119,2,0.435546,0.079058,0.097082,0.071082,0.056204,0.091344,0.014425,0.115853,0.960594
257120,3,0.385607,0.084014,0.098173,0.165116,0.119703,0.003457,0.001587,0.149515,1.007172
257121,4,0.312608,0.051079,0.188998,0.185223,0.118509,0.017475,0.001681,0.211148,1.086721


In [117]:
submit['ball_0_straight'] = submit['ball_0_straight'] / submit['total']
submit['ball_1_curve'] = submit['ball_1_curve'] / submit['total']
submit['ball_2_slider'] = submit['ball_2_slider'] / submit['total']
submit['ball_3_shoot'] = submit['ball_3_shoot'] / submit['total']
submit['ball_4_fork'] = submit['ball_4_fork'] / submit['total']
submit['ball_5_changeup'] = submit['ball_5_changeup'] / submit['total']
submit['ball_6_sinker'] = submit['ball_6_sinker'] / submit['total']
submit['ball_7_cutball'] = submit['ball_7_cutball'] / submit['total']
submit.head()

,No,ball_0_straight,ball_1_curve,ball_2_slider,ball_3_shoot,ball_4_fork,ball_5_changeup,ball_6_sinker,ball_7_cutball,total
257117,0,0.786027,0.001311,0.055769,0.091022,0.012735,0.002389,0.005838,0.044910,0.991013
257118,1,0.769813,0.010547,0.055297,0.107443,0.015066,0.005742,0.000595,0.035496,0.892985
257119,2,0.453413,0.082301,0.101064,0.073998,0.058510,0.095091,0.015017,0.120606,0.960594
257120,3,0.382862,0.083416,0.097474,0.163940,0.118850,0.003432,0.001576,0.148450,1.007172
257121,4,0.287662,0.047003,0.173916,0.170442,0.109052,0.016080,0.001547,0.194298,1.086721


In [120]:
submit.drop(['total'], axis=1, inplace=True)
submit.shape

(521650, 9)

### 2017の平均
- ストレート           0.468254
- カーブ             0.074725
- スライダー           0.185806
- シュート            0.070633
- フォーク            0.083013
- チェンジアップ         0.054217
- シンカー            0.011361
- カットボール          0.051992

In [122]:
submit.mean()

No                 260824.500000
ball_0_straight         0.487944
ball_1_curve            0.070557
ball_2_slider           0.190140
ball_3_shoot            0.066506
ball_4_fork             0.080357
ball_5_changeup         0.052917
ball_6_sinker           0.006152
ball_7_cutball          0.045428
dtype: float64

In [121]:
#submit.to_csv('submit/lgbm_1.csv', header=False, index=False)